In [ ]:
import os, glob
import random, csv
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
import pickle
from PIL import Image
import adapt
from adapt.feature_based import DANN, CORAL, DeepCORAL, CDAN
from tensorflow import keras
from keras import layers
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Input, Dense, Reshape, Dropout
from tensorflow.keras.optimizers import Adam
from keras.utils.np_utils import to_categorical

# The path of the datasets, use dict format
path = "I:/Temp/CS 766 CCPD Data"
dataset_path = {"base": path + "/ccpd/splitted_plates_base",
                "challenge": path + "/ccpd/splitted_plates_challenge",
                "db": path + "/ccpd/splitted_plates_db",
                "fn": path + "/ccpd/splitted_plates_fn",
                "weather": path + "/ccpd/splitted_plates_weather"}
img_h, img_w = 32, 16
save_check_pt = './checkpoints_Coral'


def load_csv(root, filename, name2label):
    # From csv file return images dir,labels list
    if not os.path.exists(os.path.join(root, filename)):
        images = []
        for name in name2label.keys():
            images += glob.glob(os.path.join(root, name, '*.png'))
            images += glob.glob(os.path.join(root, name, '*.jpg'))
            images += glob.glob(os.path.join(root, name, '*.jpeg'))
        #print(len(images), images)
        random.shuffle(images)  # shuffle images
        with open(os.path.join(root, filename), mode='w', newline='') as f:
            writer = csv.writer(f)
            for img in images:
                name = img.split(os.sep)[-2]
                label = name2label[name]
                writer.writerow([img, label])
            print('written into csv file:', filename)
    # read existed csv
    images, labels = [], []
    with open(os.path.join(root, filename)) as f:
        reader = csv.reader(f)
        for row in reader:
            img, label = row
            label = int(label)
            images.append(img)
            labels.append(label)
    # return img dir and label
    return images, labels


def load_ccpd(root, mode='train'):
    name2label = {}  # "sq...":0
    # iterate sub dir, sort, while keep mapping
    for name in sorted(os.listdir(os.path.join(root))):
        # skip non file folder
        if not os.path.isdir(os.path.join(root, name)):
            continue
        # give each category a number
        name2label[name] = len(name2label.keys())
    images, labels = load_csv(root, 'images.csv', name2label)
    if mode == 'train':  # 20%
        images = images[:int(0.2 * len(images))]
        labels = labels[:int(0.2 * len(labels))]
    elif mode == 'val':  # 20% = 60%->80%
        images = images[int(0.6 * len(images)):int(0.8 * len(images))]
        labels = labels[int(0.6 * len(labels)):int(0.8 * len(labels))]
    else:  # all
        images = images
        labels = labels
    return images, labels, name2label


def readCcpdImg(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L')  # conver to grayscale images
        img = img.resize([img_w, img_h])
        img_np = np.asarray(img)
        X.append(img_np.reshape([-1]))
    X = np.array(X)
    return X


def readCcpdImg_noreshape(images_dir):
    X = []
    for img_dir in images_dir:
        img = Image.open(img_dir)
        img = img.convert('L')  # conver to grayscale images
        img = img.resize([img_w, img_h])
        img_np = np.asarray(img)
        X.append(img_np)
    X = np.array(X)
    return X


def get_img_label(path, mode, num_classes=35):
    images_dir, labels, table = load_ccpd(dataset_path[path], mode=mode)
    images = readCcpdImg(images_dir)
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=num_classes)
    return images, labels


def get_encoder(input_shape=(512,)):
    model = Sequential()
    model.add(Dense(512, activation='relu',
                    input_shape=input_shape))
    model.add(Dense(256, activation="relu"))
    model.add(Dense(128, activation="sigmoid"))
    return model


def get_task(input_shape=(128,)):
    model = Sequential()
    model.add(Dense(128, activation='relu', input_shape=input_shape))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(34, activation="softmax"))
    return model


# def get_discriminator(input_shape=(128,)):
#     model = Sequential()
#     model.add(Dense(128, activation='relu', input_shape=input_shape))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
#     return model


base_images, base_labels = get_img_label('base', 'all', num_classes=34)
weather_images, weather_labels = get_img_label('weather', 'all', num_classes=34)
weather_images_t, weather_labels_t = get_img_label('weather', 'train', num_classes=34)
challenge_images, challenge_labels = get_img_label('challenge', 'all', num_classes=34)
challenge_images_t, challenge_labels_t = get_img_label('challenge', 'train', num_classes=34)
db_images, db_labels = get_img_label('db', 'all', num_classes=34)
db_images_t, db_labels_t = get_img_label('db', 'train', num_classes=34)
fn_images, fn_labels = get_img_label('fn', 'all', num_classes=34)
fn_images_t, fn_labels_t = get_img_label('fn', 'train', num_classes=34)
print("load data finished!")
from keras.applications.resnet import ResNet50
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input
from tensorflow.keras.constraints import MaxNorm
import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')
print(np.shape(base_images))
print(np.shape(base_labels))
print(np.shape(weather_images_t))
print(np.shape(weather_labels_t))

weather_model = CDAN(encoder=get_encoder(), task=get_task(),
                     lambda_=1, optimizer=Adam(0.0001), loss='CategoricalCrossentropy', metrics=["acc"], random_state=0)
weather_model.fit(base_images, base_labels, weather_images_t, weather_labels_t, epochs=80, verbose=1, batch_size=32)
weather_model.score(base_images, base_labels)
weather_model.score(weather_images, weather_labels)
weather_model.save_weights(save_check_pt+ '/weather')

challenge_model = CDAN(get_encoder(), get_task(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
challenge_model.fit(base_images, base_labels,challenge_images_t,challenge_labels_t, epochs=88,verbose=1,batch_size = 32)
challenge_model.score(base_images, base_labels)
challenge_model.score(challenge_images, challenge_labels)
challenge_model.save_weights(save_check_pt+ '/challenge')

db_model = CDAN(get_encoder(), get_task(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
db_model.fit(base_images, base_labels,db_images_t,db_labels_t, epochs=80,verbose=1,batch_size = 32)
db_model.score(base_images, base_labels)
db_model.score(db_images, db_labels)
db_model.save_weights(save_check_pt+ '/db')

fn_model = CDAN(get_encoder(), get_task(),
             lambda_=0.1, optimizer=Adam(0.0001),loss='CategoricalCrossentropy',metrics=["acc"],random_state=0)
fn_model.fit(base_images, base_labels,fn_images_t,fn_labels_t, epochs=70,verbose=1,batch_size = 32)
fn_model.score(base_images, base_labels)
fn_model.score(fn_images, fn_labels)
fn_model.save_weights(save_check_pt+ '/fn')